# 第02章：Prompts & Messages

## 本章学习目标

1. 理解Messages的作用和类型
2. 掌握ChatPromptTemplate的使用
3. 学会变量注入和模板格式化
4. 掌握Few-shot Learning技巧
5. 理解Partial Variables的应用场景
5. 学习Prompt Engineering最佳实践

## 核心概念

- **Messages**：LangChain中模型交互的基本单位
- **ChatPromptTemplate**：用于Chat Models的提示词模板
- **变量注入**：动态填充模板中的占位符
- **Few-shot Learning**：通过示例引导模型输出
- **Partial Variables**：不分变量预填充

## 本章重点

本章专注于**如何设计和组织提示词**

In [1]:
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI

# 初始化模型
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=config.OPENAI_API_KEY,
    base_url=config.OPENAI_BASE_URL,
)

print("环境准备完成")

环境准备完成


## 1. Messages基础

LangChain中的Messages是模型交互的基本单位，包含三种主要类型：

| Message类型 | 作用 | 角色标识 |
|------------|------|---------|
| **SystemMessage** | 设置模型行为和规则 | `system` |
| **HumanMessage** | 用户输入 | `user` |
| **AIMessage** | 模型回复 | `assistant` |

**为什么需要Messages？**
- Chat Models使用消息序列而非纯文本
- 明确区分不同角色的内容
- 支持多轮对话历史管理

In [2]:
# 导入Message类
from langchain.messages import SystemMessage, HumanMessage, AIMessage

# 1. 创建不同类型的Message
system_msg = SystemMessage(content="你是一个专业的Python编程助手。")
human_msg = HumanMessage(content="什么是列表推导式？")
ai_msg = AIMessage(content="列表推导式是Python中创建列表的简洁方式...")

# 2. 查看Message的属性
print("=" * 50)
print(f"SystemMessage类型：{system_msg.type}")
print(f"内容：{system_msg.content}\n")

print(f"HumanMessage类型: {human_msg.type}")
print(f"内容: {human_msg.content}\n")

print(f"AIMessage类型: {ai_msg.type}")
print(f"内容: {ai_msg.content}")
print("=" * 50)

# 3. 直接使用Messages调用模型
messages = [
    SystemMessage(content="你是一个幽默的聊天机器人。"),
    HumanMessage(content="给我讲个笑话"),
]

response = model.invoke(messages)
print(f"\n模型回复：{response.content}")
print(f"回复类型：{type(response)}")  # 返回AIMessage

SystemMessage类型：system
内容：你是一个专业的Python编程助手。

HumanMessage类型: human
内容: 什么是列表推导式？

AIMessage类型: ai
内容: 列表推导式是Python中创建列表的简洁方式...

模型回复：为什么海洋总是那么友好？

因为它有很多“波”的朋友！
回复类型：<class 'langchain_core.messages.ai.AIMessage'>


## 2. 为什么需要Prompt Templates？

在第01章中，我们直接传递字符串或消息列表给模型：

```python
# 硬编码方式
response = model.invoke("给我讲一个关于AI的笑话")
```

**问题：**
- 每次都要写完整的提示词
- 难以复用和维护
- 无法动态插入变量
- 团队协作困难

**解决方案：Prompt Templates**
- 提示词模板化
- 变量动态注入
- 代码复用性强
- 便于版本管理

## 3. ChatPromptTemplate基础

**两种创建方式：**

| 方法 | 适用场景 | 特点 |
|------|---------|------|
| `from_template()` | 简单的单条消息 | 快速创建，默认为user角色 |
| `from_messages()` | 复杂的多条消息 | 推荐使用，支持system/user/assistant |

**语法：**
- **变量占位符**：使用 `{variable_name}` 定义变量
- **调用方式**：`format_messages(variable="value")`

In [3]:
from langchain_core.prompts import ChatPromptTemplate

# ========== 方式1: from_template() ==========
# 适合简单场景，默认为user角色
simple_prompt = ChatPromptTemplate.from_template(
    "给我讲一个关于{topic}的笑话"
)

# 查看模板结构
print("【from_template 创建的模板】")
print(f"模板：{simple_prompt}")
print(f"变量：{simple_prompt.input_variables}")
print("\n" + "=" * 60 + "\n")

# 使用模板：格式化消息
messages = simple_prompt.format_messages(topic="程序员")

print("格式化后的消息：")
for msg in messages:
    print(f" [{msg.type}] {msg.content}")

# 调用模型
response = model.invoke(messages)
print(f"\nAI回复：{response.content}")

【from_template 创建的模板】
模板：input_variables=['topic'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='给我讲一个关于{topic}的笑话'), additional_kwargs={})]
变量：['topic']


格式化后的消息：
 [human] 给我讲一个关于程序员的笑话

AI回复：当然可以！这是一个关于程序员的笑话：

有一天，一个程序员走进一家咖啡店，看到菜单上有“咖啡、茶、果汁”三种饮品。他对店员说：“给我来一杯果汁。”

店员问：“你要什么水果的？”

程序员回答：“随便，反正我只会写代码，调试果汁的味道不是我的职责！”

希望你喜欢这个笑话！


In [4]:
# ========== 方式2: from_messages() - 推荐 ==========
# 支持多条消息，可以设置system、user、assistant角色

multi_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位{role}，请用专业的方式回答问题。"),
    ("user", "{question}")
])

# 查看模板结构
print("【from_messages 创建的模板】")
print(f"变量：{multi_prompt.input_variables}")
print("\n" + "=" * 60 + "\n")

# 格式化并调用
messages = multi_prompt.format_messages(
    role="Python高级工程师",
    question="什么是装饰器？用简单的语言解释"
)

print("格式化后的messages：")
for msg in messages:
    print(f" [{msg.type}] {msg.content}")

print("\n" + "="*60 + "\n")

response = model.invoke(messages)
print(f"\nAI回复：{response.content}")

【from_messages 创建的模板】
变量：['question', 'role']


格式化后的messages：
 [system] 你是一位Python高级工程师，请用专业的方式回答问题。
 [human] 什么是装饰器？用简单的语言解释



AI回复：装饰器是Python中的一个设计模式，它允许你在不修改函数或方法本身的情况下，增强或改变其功能。简单来说，装饰器就是一个接受函数作为参数并返回一个新函数的函数。

可以把装饰器想象成一个包装纸，它包裹住了一个原始的函数，给它添加了一些额外的功能，例如日志记录、权限检查或性能计时等。使用装饰器可以让代码更加清晰和可复用。

在Python中，装饰器通常使用`@decorator_name`的语法来应用，位于函数定义的上方。例如：

```python
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()
```

在这个例子中，`say_hello`函数被`my_decorator`装饰器包装，调用`say_hello()`时，会先执行装饰器中的代码，然后再执行原始的`say_hello`函数。


## 变量注入

使用`{variable_name}`语法在模板中定义占位符。

**调用时的两种方式：**

```python
# 方式1：format() - 返回字符串
formatted_str = prompt.format(topic="AI")

# 方式2：format_messages() - 返回消息列表（推荐）
messages = prompt.format_messages(topic="AI")
```

**最佳实践：**
- 变量名使用小写+下划线命名（如`user_name`)
- 调用时必须提供所有变量（除非使用Partial）
- 推荐使用`format_messages()`配合模型调用

In [5]:
# 多变量注入示例
translation_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的翻译助手，擅长{source_lang}到{target_lang}的翻译。"),
    ("user", "请将一下内容翻译成{target_lang}：\n{text}")
])

# 使用多个变量
messages = translation_prompt.format_messages(
    source_lang="中文",
    target_lang="英文",
    text="人工智能正在改变我们的生活"
)

print("格式化后的消息：")
for msg in messages:
    print(f"[{msg.type}] {msg.content}\n")

# 调用模型
response = model.invoke(messages)
print(f"\nAI翻译：{response.content}")

格式化后的消息：
[system] 你是一个专业的翻译助手，擅长中文到英文的翻译。

[human] 请将一下内容翻译成英文：
人工智能正在改变我们的生活


AI翻译：Artificial intelligence is changing our lives.


## 5. Few-shot Learning（少样本学习）

通过提供示例来引导模型输出格式和风格。

**适用场景：**
- 需要特定输出格式（如JSON、分类标签）
- 需要特定的回答风格
- 提高准确率和一致性

**结构：**
- System Message：说明任务
- Human：示例输入1
- AI：示例输出1
- Human：示例输入2
- AI：示例输出2
- ...
- Human：实际输入

**核心思想：**
展示优于描述（Show, don't tell）

In [6]:
# Few-shot示例：情感分类
sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个情感分类器，将文本分类为：积极、消极、中性"),

    # Few-shot Examples（示例）
    ("human", "这个产品太棒了！非常满意！"),
    ("ai", "积极"),

    ("human", "质量很差，完全不值这个价格。"),
    ("ai", "消极"),
    
    ("human", "就是一个普通的产品。"),
    ("ai", "中性"),

    # 实际输入
    ("human", "{text}")
])

# 测试分类
test_texts = [
    "这家餐厅的服务态度真好！",
    "等了一个小时才上菜，太慢了",
    "价格适中，味道一般"
]

print("【情感分类结果】\n")
for text in test_texts:
    messages = sentiment_prompt.format_messages(text=text)
    response = model.invoke(messages)
    print(f"文本：{text}")
    print(f"分类：{response.content}\n")

【情感分类结果】

文本：这家餐厅的服务态度真好！
分类：积极

文本：等了一个小时才上菜，太慢了
分类：消极

文本：价格适中，味道一般
分类：中性



In [7]:
# Few-shot示例：结构化输出（JSON格式）
# 注意：在模板字符串中，{{}} 用于转义真实的 {}

extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "从文本中提取关键信息，以JSON格式返回（包含name和age字段）"),
    
    # Examples - 使用 {{}} 转义JSON的 {}
    ("human", "我叫张三，今年25岁"),
    ("ai", '{{"name": "张三", "age": 25}}'),
    
    ("human", "李四的年龄是30"),
    ("ai", '{{"name": "李四", "age": 30}}'),
    
    # 实际输入
    ("human", "{text}")
])

# 测试
test_text = "王五今年28岁了"
messages = extraction_prompt.format_messages(text=test_text)
response = model.invoke(messages)

print(f"输入: {test_text}")
print(f"提取结果: {response.content}")

# 可以尝试解析JSON
import json
try:
    result = json.loads(response.content)
    print(f"解析成功: 姓名={result['name']}, 年龄={result['age']}")
except:
    print("输出不是有效的JSON（在第04章我们会学习更可靠的方法）")

输入: 王五今年28岁了
提取结果: {"name": "王五", "age": 28}
解析成功: 姓名=王五, 年龄=28


## 6. Partial Variables（部分变量预填充）

**使用场景：**
- 某些变量值在模板创建时就确定
- 需要动态生成的默认值（如当前时间）
- 减少重复传参

**两种时间：**

1. **静态预填充**：`.partial(variable="value")`
2. **动态预填充**：`.partial(variable=function)`（传入函数）

In [8]:
from datetime import datetime

# 创建基础模板
base_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是{role}, 今天是{date}"),
    ("user", "{question}")
])

print("原始模板变量：", base_prompt.input_variables)
print("\n" + "=" * 60 + "\n")

# ========== 方式1: 静态预填充 ==========
partial_prompt = base_prompt.partial(
    role="AI助手",
    date="2024年1月15日"
)

print("预填充后的变量：", partial_prompt.input_variables)

# 调用时只需提供question
messages = partial_prompt.format_messages(question="今天星期几？")

print("\n格式化后的消息：")
for msg in messages:
    print(f" [{msg.type}] {msg.content}")

response = model.invoke(messages)
print(f"\nAI回复：{response.content}")

原始模板变量： ['date', 'question', 'role']


预填充后的变量： ['question']

格式化后的消息：
 [system] 你是AI助手, 今天是2024年1月15日
 [human] 今天星期几？

AI回复：今天是2024年1月15日，星期一。


In [10]:
# ========== 方式2: 函数动态生成 ==========
def get_current_date():
    """动态获取当前日期"""
    return datetime.now().strftime("%Y年%m月%d日 %H:%M")

# 创建新的模板
date_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是{role}，当前时间：{date}"),
    ("user", "{question}")
])

# 使用函数预填充（注意：传入函数本身，不要加括号）
dynamic_prompt = date_prompt.partial(
    role="AI助手",
    date=get_current_date  # 传入函数引用
)

print("【动态预填充示例】\n")

# 每次调用时都会执行get_current_date()
messages = dynamic_prompt.format_messages(question="现在几点了？")

print("格式化后的消息:")
for msg in messages:
    print(f"  [{msg.type}] {msg.content}")

response = model.invoke(messages)
print(f"\nAI回复: {response.content}")

【动态预填充示例】

格式化后的消息:
  [system] 你是AI助手，当前时间：2025年12月17日 13:36
  [human] 现在几点了？

AI回复: 现在是2025年12月17日 13:36。


## 7. Prompt Engineering 最佳实践

### 好的Prompt特征

1. **明确具体** - 清晰说明任务和期望输出
2. **提供上下文** - 给足够的背景信息
3. **使用示例** - Few-shot Learning提高准确率
4. **结构清晰** - System/User消息职责分明
5. **设置约束** - 明确限制（长度、格式、风格）
6. **可测试** - 使用变量方便测试不同输入

### 避免的问题

1. 过于模糊的指令
2. 一次性塞入过多要求
3. 混淆System和User的职责
4. 缺少必要的约束条件
5. 没有提供示例

In [11]:
# 不好的示例
bad_prompt = ChatPromptTemplate.from_template(
    "写点关于{topic}的东西"  # 太模糊
)

# 好的示例
good_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个技术博客作者，擅长写清晰易懂的技术文章。
    
    要求：
    - 使用简介的语言
    - 包含代码示例
    - 结构清晰（标题、正文、总结）
    - 字数控制在200字以内"""),
    ("user", "请写一篇关于{topic}的技术文章")
])

# 对比测试
print("【不好的Prompt结果】")
messages1 = bad_prompt.format_messages(topic="Python装饰器")
response1 = model.invoke(messages1)
print(response1.content[:150] + "...\n")

print("=" * 60 + "\n")

print("【好的Prompt结果】")
messages2 = good_prompt.format_messages(topic="Python装饰器")
response2 = model.invoke(messages2)
print(response2.content)

【不好的Prompt结果】
Python 装饰器是 Python 中一个非常强大的特性，它允许你在不修改函数代码的情况下，动态地增加功能。装饰器本质上是一个函数，它接受一个函数作为参数，并返回一个新的函数。装饰器常用于记录日志、权限验证、性能测试等场景。

### 基本语法

装饰器的基本语法使用 `@decorator_na...


【好的Prompt结果】
# Python装饰器入门

装饰器是Python中的一个强大功能，允许你在函数或方法调用前后添加额外的功能，而无需修改原始代码。

## 什么是装饰器？

装饰器本质上是一个函数，它接受另一个函数作为参数，并返回一个新的函数。常用来进行日志记录、权限校验等操作。

## 示例代码

下面是一个简单的装饰器示例，它会在调用函数前输出日志信息：

```python
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"调用函数: {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@logger
def say_hello(name):
    print(f"你好, {name}!")

say_hello("Alice")
```

## 运行结果

当你调用`say_hello("Alice")`时，输出将是：

```
调用函数: say_hello
你好, Alice!
```

## 总结

装饰器是一个灵活而强大的工具，可以帮助你在不修改原函数的情况下，增强其功能。掌握装饰器将使你的代码更加简洁和易于维护。


## 8. 实战练习：构建智能客服Prompt

结合本章所学，创建一个完整的客服提示词模板。

**需求分析：**
- 角色设定：客服助手
- 上下文：公司信息、当前时间
- 约束条件：礼貌、简洁、专业
- 知识库：常见问题参考

In [14]:
from datetime import datetime

# 构建智能客服Prompt
customer_service_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是{company_name}的智能客服助手。
    当前时间：{current_time}
    
    工作原则：
    1. 礼貌友好，使用敬语
    2. 简洁明了，避免冗长
    3. 如果不确定，建议联系人工客服
    4. 对于投诉要表示歉意和重视
    
    常见问题参考：
    - 退货政策：7天无理由退货
    - 物流查询：提供订单号即可查询
    - 售后服务：提供1年质保"""),

    ("human", "客户问题：{customer_question}")
])

# 使用Partial预填充公司名和动态时间
service_prompt = customer_service_prompt.partial(
    company_name="科技商城",
    current_time=lambda: datetime.now().strftime("%Y年%m月%d日 %H:%M")
)

# 测试不同场景
test_questions = [
    "我想退货，怎么办理？",
    "我的订单号是123456，帮我查一下物流",
    "你们的产品质量太差了！",
]

print("【智能客服测试】\n")
for i, question in enumerate(test_questions, 1):
    print(f"场景 {i}")
    print(f"客户：{question}")

    messages = service_prompt.format_messages(customer_question=question)
    response = model.invoke(messages)

    print(f"客服：{response.content}")
    print("=" * 60 + "\n")

【智能客服测试】

场景 1
客户：我想退货，怎么办理？
客服：尊敬的客户，您好！您可以在收到商品后的7天内申请无理由退货。请您准备好订单号，并联系客服进行退货申请，我们会为您提供进一步的指导。感谢您的理解与支持！

场景 2
客户：我的订单号是123456，帮我查一下物流
客服：尊敬的客户，感谢您提供订单号。请稍等片刻，我将为您查询物流信息。 

（如果有具体的物流信息可提供，请在此处添加；如果不确定，建议告知客户联系人工客服。）

场景 3
客户：你们的产品质量太差了！
客服：非常抱歉听到您对产品质量的不满。我们非常重视您的反馈，您可以提供更多具体信息吗？这样我们可以更好地帮助您处理此问题。感谢您的理解与支持！



## 9. 复杂场景：多角色对话模板

在某些场景下，我们需要模拟完整的对话历史。

In [15]:
# 创建包含对话历史的模板
conversation_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个{role}"),
    ("human", "你好！"),
    ("ai", "你好！我是{name}，很高兴为您服务。有什么可以帮助您的吗？"),
    ("human", "{user_question}")
])

# 格式化消息
messages = conversation_prompt.format_messages(
    role="友好的购物助手",
    name="小智",
    user_question="我想买一台笔记本电脑，有什么推荐吗？"
)

print("【对话历史模板】\n")
print("格式化后的消息序列：")
for i, msg in enumerate(messages, 1):
    print(f"{i}. [{msg.type}] {msg.content}\n")

response = model.invoke(messages)
print(f"AI回复：{response.content}")

【对话历史模板】

格式化后的消息序列：
1. [system] 你是一个友好的购物助手

2. [human] 你好！

3. [ai] 你好！我是小智，很高兴为您服务。有什么可以帮助您的吗？

4. [human] 我想买一台笔记本电脑，有什么推荐吗？

AI回复：当然可以！选择笔记本电脑时，您可以考虑以下几个方面：

1. **用途**：
   - **办公和学习**：如果您主要用于文档处理、浏览网页等，推荐选择轻薄型笔记本，如联想ThinkPad系列或华为MateBook系列。
   - **游戏**：如果您想玩大型游戏，建议选择游戏本，如ROG系列或雷蛇Blade系列，这些通常配备强劲的显卡。
   - **创意工作**：如果您从事视频编辑或图形设计，可以选择高性能的MacBook Pro或戴尔XPS系列。

2. **预算**：
   - 如果预算有限，可以考虑一些性价比高的品牌，如小米笔记本或宏碁Aspire系列。
   - 如果预算充足，可以考虑高端品牌如苹果、戴尔XPS或华为MateBook X Pro。

3. **规格**：
   - **处理器**：选择Intel i5或i7，或AMD Ryzen 5或7。
   - **内存**：建议至少8GB RAM，16GB更佳。
   - **存储**：SSD更快，建议选择256GB以上的SSD。

您有特定的预算或使用需求吗？这样我可以更精准地推荐！


## 10. 高级技巧：分步思考（Chain of Thought）

引导模型逐步推理，提高复杂问题的准确率。

In [16]:
# 创建分布思考模板
cot_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个数学老师，解决问题时要：
    1. 理解问题
    2. 列出已知条件
    3. 分步骤计算
    4. 给出最终答案
    
    示例：
    问题：小明有5个苹果，小红给了他3个，他吃了2个，还剩几个？
    思考过程：
    1. 初始：5个
    2. 收到：5 + 3 = 8个
    3. 吃掉：8 - 2 = 6个
    答案：6个"""),

    ("human", "{problem}")
])

# 测试
problem = "一个班级有30名学生，其中女生占40%，后来又转来5名女生，现在女生占总人数的百分之几？"
messages = cot_prompt.format_messages(problem=problem)
response = model.invoke(messages)

print(f"问题: {problem}\n")
print(f"AI解答:\n{response.content}")

问题: 一个班级有30名学生，其中女生占40%，后来又转来5名女生，现在女生占总人数的百分之几？

AI解答:
思考过程：

1. 初始班级人数：30名学生
2. 女生占40%，所以女生人数为：30 * 40% = 30 * 0.4 = 12名女生
3. 转来5名女生后，女生人数变为：12 + 5 = 17名女生
4. 班级总人数变为：30 + 5 = 35名学生
5. 现在女生占总人数的百分比为： (17 / 35) * 100% ≈ 48.57%

最终答案：女生占总人数的约48.57%。


## 本章总结

### 核心知识点

**Messages三种类型**
- SystemMessage：设置行为
- HumanMessage：用户输入
- AIMessage：模型回复

**ChatPromptTemplate两种创建方式**
- `from_template()`：简单场景
- `from_messages()`：多消息场景（推荐）

**变量注入**
- 使用 `{variable}` 定义占位符
- `format_messages()` 返回消息列表
- 必须提供所有变量

**Few-shot Learning**
- 通过示例引导模型
- 适合特定格式输出
- 提高输出一致性

**Partial Variables**
- 静态预填充：固定值
- 动态预填充：函数生成

**最佳实践**
- 明确、具体、有示例
- 结构清晰、设置约束
- 使用分步思考处理复杂问题